In [1]:
!pip install earthengine-api --upgrade
!pip install folium

Requirement already up-to-date: earthengine-api in c:\users\schne\anaconda3\lib\site-packages (0.1.371)


In [7]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
import geopandas as gpd
import ee
# Trigger the authentication flow.
ee.Authenticate()
# Initialize the library.
ee.Initialize()

C:\Users\schne\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


Enter verification code: 4/1AfJohXmSa3twPmfz2qmqQX_MmKfFgShb3078g0Cjvklq5smVjsRUbNfEsJ8

Successfully saved authorization token.


In [8]:
gdf = gpd.read_file('buffered_stations (1).geojson')

In [9]:
# Specify the Landsat image collection
landsat = ee.ImageCollection('LANDSAT/LC08/C01/T1')

# Assume you want to get an image for the first station in the dataframe
# Create an Earth Engine geometry object from the GeoJSON geometry
aoi = ee.Geometry.Polygon(list(gdf.loc[0, 'geometry'].exterior.coords))

# Filter the Landsat collection to a single image that intersects the polygon
# and was collected closest to a certain date. Adjust the date as needed.
image = landsat.filterBounds(aoi).sort('system:time_start', False).first()

# Define the visualization parameters.
vis_params = {
  'bands': ['B5', 'B4', 'B3'], # These are the RGB bands
  'min': 0,
  'max': 500,
  'gamma': 1.4,
  }


In [17]:
import pyautogui
from datetime import datetime
import os
from ipywidgets import interact, widgets
from IPython.display import display as ipy_display, clear_output
import folium
from folium import Marker, Circle, CircleMarker
import pandas as pd
import time

# Function to save the screenshot
def save_screenshot(facility_id):
    print(f"Inside save_screenshot with Facility ID: {facility_id}")  # Debug print
    # Use the Facility ID as the filename
    filename = f"{facility_id}.jpg"
    
    directory = "C:\\screenshots"
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    # Define a path to save the screenshots. Adjust as needed.
    save_path = os.path.join("C:\\screenshots", filename)
    
    # Capture and save the screenshot
    pyautogui.screenshot(save_path, region=(1300, 641, 1700, 1710))
    print(f"Screenshot saved as {save_path}")

# Your GeoDataFrame 'gdf' should be defined somewhere here

def plot_map(desired_facility_id):
    if not desired_facility_id:  # Check if the input is empty
        print("Please enter a valid Facility ID.")
        return

    try:
        # Convert the input to integer and filter the GeoDataFrame
        facility_id = int(desired_facility_id)
        filtered_gdf = gdf[gdf['Facility ID'] == facility_id]

        # Proceed only if the Facility ID exists in the data
        if filtered_gdf.empty:
            print("Facility ID not found.")
            return

        # Extract the lat and lon from the filtered GeoDataFrame
        lat = filtered_gdf.iloc[0]['geometry'].centroid.y
        lon = filtered_gdf.iloc[0]['geometry'].centroid.x
    except ValueError:
        print("Invalid input. Please enter a numeric Facility ID.")
        return

    # Set up the map centered around the station
    m = folium.Map(location=[lat, lon], zoom_start=16, control_scale=True)

    # Add a small red dot at the center of the circle
    CircleMarker(location=[lat, lon], radius=3, color='red', fill=True, fill_color='red').add_to(m)

    # Add a circle of radius 1/4 mile (in meters) around the station
    Circle(location=[lat, lon], radius=402.336).add_to(m)

    # Add a layer control panel to the map
    m.add_child(folium.LayerControl())

    # Explicitly display the map
    ipy_display(m)

# Read the .csv file into a DataFrame
df = pd.read_csv('stations_clean_final2.csv')

# Loop through the DataFrame to get facility IDs
for index, row in df.iterrows():
    clear_output(wait=True) 
    facility_id = row['Facility ID']  # Assuming the column name is 'Facility ID'
    print(f"Processing Facility ID: {facility_id}")  # Debug print
    plot_map(facility_id)
    
    # Save the screenshot using the Facility ID as the name
    save_screenshot(facility_id)
    
    # Optional: wait for some time before the next iteration (e.g., 5 seconds)
    time.sleep(5)


Processing Facility ID: 17050


Inside save_screenshot with Facility ID: 17050
Screenshot saved as C:\screenshots\17050.jpg


In [6]:
from ipywidgets import interact, widgets
from IPython.display import display as ipy_display,  clear_output  # To avoid conflict with folium's display method
import folium
from folium import Marker, Circle, CircleMarker
import pandas as pd
import time 

# Your GeoDataFrame 'gdf' should be defined somewhere here

def plot_map(desired_facility_id):
    if not desired_facility_id:  # Check if the input is empty
        print("Please enter a valid Facility ID.")
        return

    try:
        # Convert the input to integer and filter the GeoDataFrame
        facility_id = int(desired_facility_id)
        filtered_gdf = gdf[gdf['Facility ID'] == facility_id]

        # Proceed only if the Facility ID exists in the data
        if filtered_gdf.empty:
            print("Facility ID not found.")
            return

        # Extract the lat and lon from the filtered GeoDataFrame
        lat = filtered_gdf.iloc[0]['geometry'].centroid.y
        lon = filtered_gdf.iloc[0]['geometry'].centroid.x
    except ValueError:
        print("Invalid input. Please enter a numeric Facility ID.")
        return

    # Set up the map centered around the station
    m = folium.Map(location=[lat, lon], zoom_start=16, control_scale=True)

    # Add a small red dot at the center of the circle
    CircleMarker(location=[lat, lon], radius=3, color='red', fill=True, fill_color='red').add_to(m)

    # Add a circle of radius 1/4 mile (in meters) around the station
    Circle(location=[lat, lon], radius=402.336).add_to(m)

    # Add the satellite image as an overlay (uncomment the following lines if you have the image details)
    # mapid = image.getMapId(vis_params)
    # folium.TileLayer(
    #     tiles=mapid['tile_fetcher'].url_format,
    #     attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    #     overlay=True,
    #     name='median composite',
    # ).add_to(m)

    # Add a layer control panel to the map
    m.add_child(folium.LayerControl())

    # Explicitly display the map
    ipy_display(m)

# Read the .csv file into a DataFrame
df = pd.read_csv('stations_clean_final2.csv')

# Loop through the DataFrame to get facility IDs
for index, row in df.iterrows():
    clear_output(wait=True) 
    facility_id = row['Facility ID']  # Assuming the column name is 'Facility ID'
    plot_map(facility_id)
    
    time.sleep(5)  # Wait for 5 seconds before showing the next map

KeyboardInterrupt: 

In [ ]:
# Read the .csv file into a DataFrame
df = pd.read_csv('path/to/your/csv/file.csv')

# Loop through the DataFrame to get facility IDs
for index, row in df.iterrows():
    facility_id = row['Facility ID']  # Assuming the column name is 'Facility ID'
    plot_map(facility_id)
    
    time.sleep(5)  # Wait for 5 seconds before showing the next map
    
    # Create text box widget for Facility ID input
facility_id_input = widgets.Text(
    value='',
    placeholder='Enter Facility ID',
    description='Facility ID:',
    disabled=False
)

# Display the text box and map based on entered Facility ID
interact(plot_map, desired_facility_id=facility_id_input)

In [ ]:
import pyautogui

In [46]:
# Take a screenshot of a specific region: (x=100, y=100, width=300, height=400)
screenshot = pyautogui.screenshot(region=(1400, 691, 2500, 1800))

In [47]:
screenshot.save('6689.png')

In [48]:
from PIL import Image, ImageDraw

# Open the existing screenshot
screenshot_path = '6689.png'  # Replace this with the path to your screenshot
screenshot = Image.open(screenshot_path)

# Create a mask of the same size as the screenshot
width, height = screenshot.size
mask = Image.new('L', (width, height), 0)
draw = ImageDraw.Draw(mask)

# Coordinates of the circle's center within the screenshot
circle_center_x, circle_center_y = 1219, 734  # Obtained from earlier analysis

# Use the previously calculated radius but adjust for the right side
circle_radius = 1219  # Obtained from earlier analysis
right_side_adjustment = 1000  # Amount to reduce the radius on the right side


# Draw an adjusted ellipse (circle with reduced right side)
draw.ellipse((circle_center_x - circle_radius, circle_center_y - circle_radius, 
              circle_center_x + circle_radius - right_side_adjustment, circle_center_y + circle_radius), fill=255)

# Create a new image to store the circular screenshot
circular_screenshot = Image.new('RGB', (width, height))

# Paste the original screenshot using the mask
circular_screenshot.paste(screenshot, (0, 0), mask=mask)

# Save the circular screenshot
circular_screenshot.save('circular_6689.png')


In [ ]:
from PIL import ImageGrab
import time

while True:
    # Capture the screen
    img = ImageGrab.grab()
    # Save the image
    img.save("screenshot.png")
    
    # Wait for 5 seconds before the next capture
    time.sleep(5)
